In [1]:
import pandas as pd
data = pd.read_excel('hi_3500.ods', engine='odf')
data.head()

,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान जाएगा तू ले जाकर दिल्ली इसे दिखा ला दोस्त,negative
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
4,कश्मीर में हो रहा है जल जिहाद ।,negative


In [2]:
data.shape

(9076, 2)

In [3]:
data.rename(columns={'लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान जाएगा तू ले जाकर दिल्ली इसे दिखा ला दोस्त': 'text', 'negative': 'sentiment'}, inplace=True)
data.head()

,text,sentiment
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
4,कश्मीर में हो रहा है जल जिहाद ।,negative


In [4]:
data['text'][:10]

0    गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...
1    ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है
2    यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...
3    अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...
4                      कश्मीर में हो रहा है जल जिहाद ।
5    परमपिता परमेश्वर अगर हिंदुस्तान मेँ दंगा होता ...
6    संघर्ष में आदमी अकेला होता है सफलता में दुनिया...
7                                 मोहब्बत अंधी होती है
8    तीखी बात में राम जेठमलानी ने गड्गरी पर तो इल्ज...
9    एक मुलाकात जरूरी है रिहा होने के लिये ।एक उम्र...
Name: text, dtype: object

In [5]:
print(set(data['sentiment'].values))

{'neutral', 'positive', 'negative'}


In [6]:
data.sentiment = pd.Categorical(data.sentiment)
data['labels'] = data['sentiment'].cat.codes
data = data.dropna()

In [7]:
label_encoding_dict = dict(zip(data['sentiment'], data['labels']))
print(label_encoding_dict)

{'negative': 0, 'positive': 2, 'neutral': 1}


In [8]:
data.head()

,text,sentiment,labels
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative,0
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative,0
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative,0
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative,0
4,कश्मीर में हो रहा है जल जिहाद ।,negative,0


In [9]:
data = data.drop(['sentiment'], axis=1)
data.head()

,text,labels
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,0
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,0
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,0
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,0
4,कश्मीर में हो रहा है जल जिहाद ।,0


In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=91)

In [11]:
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['train'] = tds
dataset['validation'] = vds

print(dataset)

/home/amogh/Downloads/Projects/hindi-bert/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 6807
    })
    validation: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 2269
    })
})


In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

2023-09-27 08:22:25.349259: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 08:22:29.083873: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 08:22:34.224797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
# model = AutoModelForSequenceClassification.from_pretrained('monsoon-nlp/hindi-tpu-electra', num_labels=3)
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

In [14]:
def process_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_data = dataset.map(process_function, batched=True)

Map: 100%|██████████| 2269/2269 [00:00<00:00, 26022.87 examples/s]


In [15]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6807
    })
    validation: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2269
    })
})

In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [22]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [23]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [24]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

2023-09-27 08:22:46.973070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 08:22:46.973451: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [25]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "google/muril-base-cased", num_labels=3, id2label=id2label, label2id=label2id
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
from transformers.keras_callbacks import KerasMetricCallback

model.compile(optimizer=optimizer)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [28]:
callbacks = [metric_callback]
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=callbacks)

Epoch 1/5
425/425 [==============================] - 1343s 3s/step - loss: 0.8344 - val_loss: 0.5832 - accuracy: 0.8391
Epoch 2/5
425/425 [==============================] - 1311s 3s/step - loss: 0.4655 - val_loss: 0.3563 - accuracy: 0.8854
Epoch 3/5
425/425 [==============================] - 1302s 3s/step - loss: 0.2709 - val_loss: 0.2598 - accuracy: 0.9193
Epoch 4/5
425/425 [==============================] - 1309s 3s/step - loss: 0.1763 - val_loss: 0.2508 - accuracy: 0.9233
Epoch 5/5
425/425 [==============================] - 1301s 3s/step - loss: 0.1302 - val_loss: 0.2522 - accuracy: 0.9242


In [33]:
text = "आजकल की तनावपूर्ण ज़िंदगी में सुख कहाँ है? बस चिंता, तनाव, और परेशानी ही हर दिन का हिस्सा बन गए हैं"
inputs = tokenizer(text, return_tensors='tf')
outputs = model(**inputs)
print(outputs)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 2.9032602 , -1.969286  , -0.45796463]], dtype=float32)>, hidden_states=None, attentions=None)


In [34]:
logits = outputs.logits
print(logits)

tf.Tensor([[ 2.9032602  -1.969286   -0.45796463]], shape=(1, 3), dtype=float32)


In [35]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'negative'

In [36]:
print(predicted_class_id)

0


In [37]:
import pickle

model_filename = 'hindi-sentiment-model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

/home/amogh/Downloads/Projects/hindi-bert/test/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
